# Demo: VCD-Net training 

This notebook demonstrates how to train a VCD-Net on a given dataset, which is assumed to be prepared by [datapre notebook](../datapre/datapre.ipynb) in our datapre package. 

### Notes on dataset in this demo
After you go through [datapre notebook](../datapre/datapre.ipynb), the dataset should be stored in */results/TrainingPair*. We also provide a dataset generated in advance [here](https://drive.google.com/file/d/1h_Q7ylHeMh9dCUeo8Fz8o2j0WsM25-2g/view?usp=sharing) (under data/train/), in case you want to skip the data preparation step. By default, this notebook assumes that the data for training has been located under *./data/train*. You can always change the file path in the [configuration script](./config.py) to alter the target dataset though. 



## Examine the parameters
We define all parameters in a [configuration script](./config.py). Make the edit according to specific application and dataset before running this notebook.  

Typical parameters include:
- **img_size** : the lateral pixel size of the input patch, assuming it's square 
- **PSF.n_slices** : number of z slices of targeted 3-D reconstruction
- **PSF.Nnum**     : number of pixels behind each lenslet
- **label**        : label for folder naming
- **TRAIN.target3d_path** : folder where the 3D HR patches are stored 
- **TRAIN.lf2d_path**     : folder where the 2D light field patches are stored
- **TRAIN.n_epoch**       : number of epochs for training 

In [1]:
from dataset import Dataset
from train import Trainer
from config import config
import warnings
warnings.filterwarnings('ignore')

print("Parameters defined in config.py:")
for par, val in config.items():
    if not type(config[par]) == type(config):
        print('    {:<30}   {:}'.format(par,val))

print("PSF related: ")
for par, val in config.PSF.items():
    print('    {:<30}   {:<30}'.format(par,val))
    
print("Training related: ")
for par, val in config.TRAIN.items():
    print('    {:<30}   {:<30}'.format(par,val))
    



Parameters defined in config.py:
    size_factor                      [1, 1]
    img_size                         176
    label                            tubulin_40x_n11_[m30-30]_step1um_xl9_num120_sparse
    model                            structure
    n_channels                       1
PSF related: 
    Nnum                             11                            
    n_slices                         61                            
Training related: 
    target3d_path                    ../../data/TrainingPair/WF/   
    valid_on_the_fly                 0                             
    log_dir                          ./log/tubulin_40x_n11_[m30-30]_step1um_xl9_num120_sparse/
    lr_decay                         0.1                           
    using_edge_loss                  0                             
    device                           0                             
    lr_init                          0.0001                        
    beta1                         

## Train the VCD-Net
### Import the dataset

In [2]:
import numpy as np

img_size         = config.img_size * np.array(config.size_factor) 
n_num            = config.PSF.Nnum
base_size        = img_size // n_num # lateral size of lf_extra
training_dataset = Dataset(config.TRAIN.target3d_path, config.TRAIN.lf2d_path, config.PSF.n_slices, 
                           config.PSF.Nnum, base_size, normalize_mode='max')

### Start training
**This step takes long time.** Due to limitation in environment setup on code ocean (when dealing with both Matlab and Tensorflow), we trade-off to **use CPU** for this demonstration. Note that you can use GPU for better performance on your local machine, which we highly recommend. It's encouraged to download the code and test locally to get away with all the limitation on the cloud service.

**Important:** For simply reviewing our VCD-LFM pipeline, we suggest you **interrupt** the following training process and directly use the checkpoint we uploaded in */data/checkpoint/* and navigate to the [predict notebook](./predict_demo.ipynb).

Try restart the kernel for unpredicted error in this step. 

In [3]:
use_cpu = True # set to False if a CUDA enabled GPU is available. Note: GPU is not supported in current code ocean environment

trainer  = Trainer(training_dataset)
trainer.build_graph(use_cpu)
trainer.train(begin_epoch=0)

[TL] InputLayer  vcdnet/lf_extra: (1, 16, 16, 121)
[TL] Conv2dLayer vcdnet/conv1: shape:(7, 7, 121, 128) strides:(1, 1, 1, 1) pad:SAME act:identity



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[TL] SubpixelConv2d  interp/subpixel0: scale: 2 n_out_channel: 32 act: identity
[TL] Conv2dLayer vcdnet/interp/conv0: shape:(3, 3, 32, 64) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] SubpixelConv2d  interp/subpixel1: scale: 2 n_out_channel: 16 act: identity
[TL] Conv2dLayer vcdnet/interp/conv1: shape:(3, 3, 16, 32) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] SubpixelConv2d  interp/subpixel2: scale: 2 n_out_channel: 8 act: identity
[TL] Conv2dLayer vcdnet/interp/conv2: shape:(3, 3, 8, 16) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] SubpixelConv2d  interp/subpixel3: scale: 2 n_out_channel: 4 act: identity
[TL] Conv2dLayer vcdnet/interp/conv3: shape:(3, 3, 4, 8) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] Conv2dLayer vcdnet/interp/conv_final: shape:(3, 3, 8, 8) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] BatchNormLayer vcdnet/interp/bn_final: decay:0.900000 epsilon:0.000010 act:identity is_train:True
[TL] Conv2dLayer vcdnet/encoder/conv0: shape:(3, 3, 8, 64) strides:(1

encoder 0 : (1, 256, 256, 64)


[TL] Conv2dLayer vcdnet/encoder/conv1: shape:(3, 3, 64, 128) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] BatchNormLayer vcdnet/encoder/bn1: decay:0.900000 epsilon:0.000010 act:identity is_train:True
[TL] ElementwiseLayer vcdnet/encoder/add1: size:(1, 256, 256, 128) fn:add
[TL] PoolLayer   vcdnet/encoder/maxplool1: ksize:[1, 3, 3, 1] strides:[1, 2, 2, 1] padding:SAME pool:max_pool
[TL] Conv2dLayer vcdnet/encoder/conv2: shape:(3, 3, 128, 256) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] BatchNormLayer vcdnet/encoder/bn2: decay:0.900000 epsilon:0.000010 act:identity is_train:True
[TL] ElementwiseLayer vcdnet/encoder/add2: size:(1, 128, 128, 256) fn:add
[TL] PoolLayer   vcdnet/encoder/maxplool2: ksize:[1, 3, 3, 1] strides:[1, 2, 2, 1] padding:SAME pool:max_pool
[TL] Conv2dLayer vcdnet/encoder/conv3: shape:(3, 3, 256, 512) strides:(1, 1, 1, 1) pad:SAME act:identity
[TL] BatchNormLayer vcdnet/encoder/bn3: decay:0.900000 epsilon:0.000010 act:identity is_train:True
[TL] ElementwiseLayer

...

decoder 0 : (1, 256, 256, 128)


[TL] BatchNormLayer vcdnet/decoder/bn6: decay:0.900000 epsilon:0.000010 act:identity is_train:True
[TL] UpSampling2dLayer upsamplimg6: is_scale:False size:(256, 256) method:0 align_corners:False
[TL] UpSampling2dLayer resize_final: is_scale:False size:[176 176] method:0 align_corners:False
[TL]   param   0: vcdnet/conv1/W_conv2d:0 (7, 7, 121, 128)    float32_ref
[TL]   param   1: vcdnet/interp/conv0/W_conv2d:0 (3, 3, 32, 64)     float32_ref
[TL]   param   2: vcdnet/interp/conv1/W_conv2d:0 (3, 3, 16, 32)     float32_ref
[TL]   param   3: vcdnet/interp/conv2/W_conv2d:0 (3, 3, 8, 16)      float32_ref
[TL]   param   4: vcdnet/interp/conv3/W_conv2d:0 (3, 3, 4, 8)       float32_ref
[TL]   param   5: vcdnet/interp/conv_final/W_conv2d:0 (3, 3, 8, 8)       float32_ref
[TL]   param   6: vcdnet/interp/bn_final/beta:0 (8,)               float32_ref
[TL]   param   7: vcdnet/interp/bn_final/gamma:0 (8,)               float32_ref
[TL]   param   8: vcdnet/interp/bn_final/moving_mean:0 (8,)            

[TL] [*] creates ./sample/test/tubulin_40x_n11_[m30-30]_step1um_xl9_num120_sparse/ ...
[TL] [*] creates ./checkpoint/tubulin_40x_n11_[m30-30]_step1um_xl9_num120_sparse/ ...
[TL] [!] ./log/tubulin_40x_n11_[m30-30]_step1um_xl9_num120_sparse/ exists ...


040-000063.tif : (176, 176, 61)
040-000063.tif : (16, 16, 121)
HR dataset : (2358, 176, 176, 61)
LF dataset: (2358, 16, 16, 121)

Epoch:[0/200] iter:[5/2123] time: 4.174s, {'ln_loss': 0.8491743}}

AttributeError: 'Trainer' object has no attribute 'test_loss_plt'

## What's next
Now we have a trained model. Navigate to the [predict notebook](./predict_demo.ipynb) to apply it to reconstruct a light field raw image into 3D stack.